In [1]:
# Import statements
import sys
import numpy as np
import random

In [2]:
# Debug function
def debug(expression):
    frame = sys._getframe(1)
    value = str(eval(expression, frame.f_globals, frame.f_locals))
    if '\n' in value:
        delimiter = ':\n'
    else:
        delimiter = ' ='
    print(expression + delimiter, value)
    

In [3]:
class NeuralNetwork:
    def __init__(self, input_nodes, hidden_nodes, output_nodes):
        self.layer_0_nodes = input_nodes
        self.layer_1_nodes = hidden_nodes
        self.layer_2_nodes = output_nodes

        # Seed
        np.random.seed(1)

        # Init weights
        self.weights_0_1 = np.random.normal(0.0, self.layer_1_nodes**-0.5, (self.layer_0_nodes, self.layer_1_nodes))
        self.weights_1_2 = np.random.normal(0.0, self.layer_2_nodes**-0.5, (self.layer_1_nodes, self.layer_2_nodes))


    @staticmethod
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    @staticmethod
    def sigmoid_output_2_derivative(output):
        return output * (1 - output)

    def train(self, inputs, targets, iterations, learn_rate=0.01):
        assert(len(inputs) == len(targets))
        for iteration in range(iterations):

            # self.weights_0_1.shape # 3x4
            # self.weights_1_2.shape # 4x1

            # inputs.shape # 6x3
            # targets.shape # 6x1

            # Forward Pass #
            hidden_input = inputs.dot(self.weights_0_1)  # 6x4
            hidden_output = self.sigmoid(hidden_input)  # 6x4

            final_input = hidden_output.dot(self.weights_1_2)  # 6x1
            final_output = self.sigmoid(final_input)  # 6x1

            # Back Pass #
            final_error = targets - final_output  # 6x1
            final_grad = final_error * self.sigmoid_output_2_derivative(final_output)  # 6x1

            hidden_error = final_error.dot(self.weights_1_2.T)  # 6x4
            hidden_grad = hidden_error * self.sigmoid_output_2_derivative(hidden_output)  # 6x4

            # Update Weights #
            self.weights_1_2 += hidden_output.T.dot(final_grad) * learn_rate
            self.weights_0_1 += inputs.T.dot(hidden_grad) * learn_rate

    def predict(self, inputs):
        # Forward Pass #
        hidden_input = inputs.dot(self.weights_0_1)  # 6x4
        hidden_output = self.sigmoid(hidden_input)  # 6x4

        final_input = hidden_output.dot(self.weights_1_2)  # 6x1
        final_output = self.sigmoid(final_input)  # 6x1
        return final_output


In [4]:
def train_test_split(features, labels, test_size=0.25, random_state=1):
    
    assert(len(features) == len(labels))
    assert(len(features) > 1)
    combined = list(zip(features, labels))
    random.seed(random_state)
    random.shuffle(combined)
    
    features[:], labels[:] = zip(*combined)
    
    split_idx = int(np.ceil(len(features) * test_size))
    
    features_train, features_test, labels_train, labels_test = \
        features[split_idx:], features[:split_idx], labels[split_idx:], labels[:split_idx]
    
    return features_train, features_test, labels_train, labels_test

In [5]:
def accuracy_score(logits, labels):
    assert(len(logits) == len(labels))
    assert(len(logits) > 0)
    
    correct = 0
    
    for logit, label in zip(logits, labels):
        if logit == label:
            correct += 1
    return correct/len(logits)


In [20]:
input_nodes = 6
hidden_nodes = 4
output_nodes = 1 # don't change
function = lambda features: np.sin(np.sum(features)) > 0.5
learn_rate = 0.01
epochs = 100000
test_split = 0.10

# Print
print_sample = 10
decimal_round = 5
spacing = 5

In [21]:

# Create network
network = NeuralNetwork(input_nodes, hidden_nodes, 1)

# Generate data
features = np.array(np.meshgrid( *([[0, 1]]*input_nodes) )).T.reshape(-1,input_nodes)
labels = np.zeros((features.shape[0], output_nodes), dtype=int)
for key, row in enumerate(features):
    if (function(row)): # Qualifier
        labels[key] = 1
        
# Split Data
features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=test_split, random_state=42)

# Train Network
network.train(features_train, labels_train, epochs, learn_rate=learn_rate)

# Predict
predictions_test = network.predict(features_test)

# Print
format_string = '{:'+str(input_nodes*2+1+spacing)+'}{:'+str((decimal_round+2)*output_nodes+3+spacing)+'}{:'+str(output_nodes*2+1+spacing)+'}{}'   

print(format_string.format('Features', 'Preds', 'Labels', 'Correct'))
# Show sample
for feature, prediction, label in zip(features_test[:print_sample], predictions_test[:print_sample], labels_test[:print_sample]):
    correct = np.array_equal(np.round(prediction), label)
    print(format_string.format(str(feature), str(np.round(prediction, decimals=decimal_round)), str(label), str(correct)))

# Show accuracy
accuracy = accuracy_score(labels_test, np.round(predictions_test))
print('\nAccuracy:', accuracy)


Features          Preds          Labels  Correct
[1 1 1 0 1 0]     [ 0.00671]     [0]     True
[0 1 0 1 1 0]     [ 0.00833]     [0]     True
[1 1 1 0 0 0]     [ 0.00671]     [0]     True
[1 0 1 0 1 0]     [ 0.00794]     [0]     True
[0 1 1 1 0 1]     [ 0.00681]     [0]     True
[0 1 0 0 0 1]     [ 0.98852]     [1]     True
[1 1 0 0 1 0]     [ 0.00704]     [0]     True

Accuracy: 1.0
